In [6]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import numpy as np
import xgboost as xgb

xgb_dir = "/home/yannick/github/xgboost/demo"

### load data in do training
dtrain = xgb.DMatrix(xgb_dir + '/data/agaricus.txt.train')
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}

num_round = 2

In [33]:
def hyperopt_train_test(params):

    params["objective"] = 'binary:logistic'
    params["silent"] = 1
    params['max_depth'] = 1+ params["max_depth"]
    
    print params
    
    r = xgb.cv(params, dtrain, num_round, nfold=5,
               metrics={'error'})

    return r.tail(1)["train-error-mean"]

space4xgb = {
    'max_depth': hp.choice('max_depth', range(1, 3)),
    'eta':       hp.loguniform('eta', 0.001, 0.5)
}

def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4xgb, algo=tpe.suggest, max_evals=20, trials=trials)
print 'best:'
print best
print hyperopt_train_test(best)

{'objective': 'binary:logistic', 'eta': 1.2925900627416496, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.4608767580619824, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.4097813844177456, 'max_depth': 3, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.1757658337235832, 'max_depth': 3, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.5346156380443898, 'max_depth': 3, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.5331992974411357, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.2639768544138, 'max_depth': 3, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.321422808627276, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.614118495349361, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.1656367493467408, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logistic', 'eta': 1.1265110404255056, 'max_depth': 2, 'silent': 1}
{'objective': 'binary:logi

In [34]:
for t in trials:
    print t['misc']['vals'], '->\t', t['result']['loss']

{'eta': [1.2925900627416496], 'max_depth': [0]} ->	0.0150154
{'eta': [1.4608767580619824], 'max_depth': [0]} ->	0.0150154
{'eta': [1.4097813844177456], 'max_depth': [1]} ->	0.0102152
{'eta': [1.1757658337235832], 'max_depth': [1]} ->	0.0006912
{'eta': [1.5346156380443898], 'max_depth': [1]} ->	0.009908
{'eta': [1.5331992974411357], 'max_depth': [0]} ->	0.0150154
{'eta': [1.2639768544138], 'max_depth': [1]} ->	0.0058368
{'eta': [1.321422808627276], 'max_depth': [0]} ->	0.0150154
{'eta': [1.614118495349361], 'max_depth': [0]} ->	0.0150154
{'eta': [1.1656367493467408], 'max_depth': [0]} ->	0.0195082
{'eta': [1.1265110404255056], 'max_depth': [0]} ->	0.0213132
{'eta': [1.6166477472161582], 'max_depth': [0]} ->	0.0150154
{'eta': [1.5072707503183662], 'max_depth': [1]} ->	0.0102152
{'eta': [1.4266650628398512], 'max_depth': [0]} ->	0.0150154
{'eta': [1.1166264611949077], 'max_depth': [1]} ->	0.0009216
{'eta': [1.4453244612088565], 'max_depth': [1]} ->	0.0102152
{'eta': [1.532324803923355], '

In [24]:
range(1,20)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]